In [1]:
d1 = spark.read.csv('/user/maxnet/database/sig.db/term_desc_all',sep='\x01')
d1 = d1.withColumnRenamed('_c0','term_id')\
       .withColumnRenamed('_c1','brand')\
       .withColumnRenamed('_c2','cn_name')\
       .withColumnRenamed('_c3','en_name')\
       .withColumnRenamed('_c4','type1')\
       .withColumnRenamed('_c5','type2')\
       .withColumnRenamed('_c6','os')\
       .withColumnRenamed('_c7','dtime')\
       .withColumnRenamed('_c8','price')\
       .withColumnRenamed('_c9','remarks')\
       .withColumnRenamed('_c10','prio')

d2 = spark.read.csv('/user/maxnet/database/sig.db/term_oui',sep='\x01')
d2 = d2.withColumnRenamed('_c0','mac_6').withColumnRenamed('_c1','manu')

from pyspark.sql.functions import *
d = spark.read.csv('/user/maxnet/database/sig.db/data_mac_res',sep='\x01')
d = d.withColumnRenamed('_c0','mac').withColumnRenamed('_c1','id')
d = d.withColumn('m1_6',substring(d.mac,1,6))

tmp_1 = d.join(d2,d.m1_6==d2.mac_6)
df = tmp_1.join(d1,tmp_1.id == d1.term_id).select('mac','id','manu','brand','prio').distinct().dropna()
df = df.filter(df.prio == 4).select('mac','id','brand').distinct().dropna()
df.write.mode('overwrite').parquet('hdfs:///data/user/hive/warehouse/ian/distance_dataframe',compression='gzip')

In [2]:
df = spark.read.parquet('/data/user/hive/warehouse/ian/distance_dataframe')
df.select('mac').distinct().count(),df.select('id').distinct().count(),df.select('brand').distinct().count()

(199475790, 4082, 287)

In [3]:
df = df.withColumn('m1',substring(df.mac,1,6)).withColumn('m2',substring(df.mac,7,6))

In [4]:
df = df.withColumn('p',conv(df.m2, 16, 10))

In [5]:
df = df.select('mac','id','m1','m2',col('p').cast('bigint'))

In [6]:
df.select('m1').distinct().sample(0.1).show(10,truncate=False)

+------+
|m1    |
+------+
|7C1C68|
|0018AC|
|001A84|
|3C8C40|
|0019B1|
|00502B|
|00D056|
|0014A9|
|0080F6|
|001438|
+------+
only showing top 10 rows



In [6]:
d1 = df.filter(df.m1 =='24DF6A')

In [7]:
d1.select('id').distinct().count()

309

In [8]:
d1.count()

125692

In [10]:
d1.sort('p').show(truncate=False)

+------------+-----+------+------+----+
|mac         |id   |m1    |m2    |p   |
+------------+-----+------+------+----+
|24DF6A001848|40316|24DF6A|001848|6216|
|24DF6A001856|40316|24DF6A|001856|6230|
|24DF6A00185E|40316|24DF6A|00185E|6238|
|24DF6A001866|40316|24DF6A|001866|6246|
|24DF6A001873|40316|24DF6A|001873|6259|
|24DF6A001880|40316|24DF6A|001880|6272|
|24DF6A001884|40316|24DF6A|001884|6276|
|24DF6A001891|40316|24DF6A|001891|6289|
|24DF6A001892|40316|24DF6A|001892|6290|
|24DF6A001899|40316|24DF6A|001899|6297|
|24DF6A0018A3|40316|24DF6A|0018A3|6307|
|24DF6A0018A7|40316|24DF6A|0018A7|6311|
|24DF6A0018BC|40316|24DF6A|0018BC|6332|
|24DF6A0018BD|40316|24DF6A|0018BD|6333|
|24DF6A0018D9|40316|24DF6A|0018D9|6361|
|24DF6A001907|40316|24DF6A|001907|6407|
|24DF6A001908|40316|24DF6A|001908|6408|
|24DF6A001914|40316|24DF6A|001914|6420|
|24DF6A00191E|40316|24DF6A|00191E|6430|
|24DF6A001928|40316|24DF6A|001928|6440|
+------------+-----+------+------+----+
only showing top 20 rows



In [71]:
#test_mac : 24DF6A0018CA 6346
#           24DF6A0018CE 6350

from pyspark.sql.types import *
test1 = [{'mac':'24DF6A0018CA'}]
test2 = [{'mac':'24DF6A0018CE'}]
inp1 = spark.createDataFrame(test1)
inp2 = spark.createDataFrame(test2)
inp = inp1.union(inp2)
inp = inp.withColumn('m1',substring(inp.mac,1,6)).withColumn('m2',substring(inp.mac,7,6)).withColumn('id',lit(None).cast(StringType()))
inp = inp.withColumn('p',conv(inp.m2, 16, 10))  
inp = inp.select('mac','m1','m2',col('p').cast('bigint'),'id')

In [72]:
#low = d1.select(min(col('p'))).collect()[0]['min(p)'] 
#up = d1.select(max(col('p'))).collect()[0]['max(p)']

In [73]:
inp.show(truncate=False)

+------------+------+------+----+----+
|mac         |m1    |m2    |p   |id  |
+------------+------+------+----+----+
|24DF6A0018CA|24DF6A|0018CA|6346|null|
|24DF6A0018CE|24DF6A|0018CE|6350|null|
+------------+------+------+----+----+



In [74]:
inp.dtypes

[('mac', 'string'),
 ('m1', 'string'),
 ('m2', 'string'),
 ('p', 'bigint'),
 ('id', 'string')]

In [75]:
combine = inp.unionByName(d1).sort('p',ascending=True)

In [76]:
combine.show(truncate=False)

+------------+------+------+----+-----+
|mac         |m1    |m2    |p   |id   |
+------------+------+------+----+-----+
|24DF6A001848|24DF6A|001848|6216|40316|
|24DF6A001856|24DF6A|001856|6230|40316|
|24DF6A00185E|24DF6A|00185E|6238|40316|
|24DF6A001866|24DF6A|001866|6246|40316|
|24DF6A001873|24DF6A|001873|6259|40316|
|24DF6A001880|24DF6A|001880|6272|40316|
|24DF6A001884|24DF6A|001884|6276|40316|
|24DF6A001891|24DF6A|001891|6289|40316|
|24DF6A001892|24DF6A|001892|6290|40316|
|24DF6A001899|24DF6A|001899|6297|40316|
|24DF6A0018A3|24DF6A|0018A3|6307|40316|
|24DF6A0018A7|24DF6A|0018A7|6311|40316|
|24DF6A0018BC|24DF6A|0018BC|6332|40316|
|24DF6A0018BD|24DF6A|0018BD|6333|40316|
|24DF6A0018CA|24DF6A|0018CA|6346|null |
|24DF6A0018CE|24DF6A|0018CE|6350|null |
|24DF6A0018D9|24DF6A|0018D9|6361|40316|
|24DF6A001907|24DF6A|001907|6407|40316|
|24DF6A001908|24DF6A|001908|6408|40316|
|24DF6A001914|24DF6A|001914|6420|40316|
+------------+------+------+----+-----+
only showing top 20 rows



In [77]:
combine = combine.withColumn('index',monotonically_increasing_id())

In [78]:
combine.show(truncate=False)

+------------+------+------+----+-----+-----+
|mac         |m1    |m2    |p   |id   |index|
+------------+------+------+----+-----+-----+
|24DF6A001848|24DF6A|001848|6216|40316|0    |
|24DF6A001856|24DF6A|001856|6230|40316|1    |
|24DF6A00185E|24DF6A|00185E|6238|40316|2    |
|24DF6A001866|24DF6A|001866|6246|40316|3    |
|24DF6A001873|24DF6A|001873|6259|40316|4    |
|24DF6A001880|24DF6A|001880|6272|40316|5    |
|24DF6A001884|24DF6A|001884|6276|40316|6    |
|24DF6A001891|24DF6A|001891|6289|40316|7    |
|24DF6A001892|24DF6A|001892|6290|40316|8    |
|24DF6A001899|24DF6A|001899|6297|40316|9    |
|24DF6A0018A3|24DF6A|0018A3|6307|40316|10   |
|24DF6A0018A7|24DF6A|0018A7|6311|40316|11   |
|24DF6A0018BC|24DF6A|0018BC|6332|40316|12   |
|24DF6A0018BD|24DF6A|0018BD|6333|40316|13   |
|24DF6A0018CA|24DF6A|0018CA|6346|null |14   |
|24DF6A0018CE|24DF6A|0018CE|6350|null |15   |
|24DF6A0018D9|24DF6A|0018D9|6361|40316|16   |
|24DF6A001907|24DF6A|001907|6407|40316|17   |
|24DF6A001908|24DF6A|001908|6408|4

In [79]:
#p_min = combine.filter(combine.id.isNull()).select(min('index')).collect()[0]['min(index)']
#p_max = combine.filter(combine.id.isNull()).select(max('index')).collect()[0]['max(index)']

In [80]:
combine.select(max('p')).show()

+--------+
|  max(p)|
+--------+
|16777078|
+--------+



In [131]:
gp_id_pmin = combine.groupBy('id').agg({'p':'min'})
gp_id_pmax = combine.groupBy('id').agg({'p':'max'}).withColumnRenamed('id','id1')
gp_id_count = combine.groupBy('id').count().withColumnRenamed('id','id2')

stat = gp_id_pmin.join(gp_id_pmax,gp_id_pmin.id ==gp_id_pmax.id1)
stat = stat.join(gp_id_count,stat.id == gp_id_count.id2)
stat = stat.select('id','min(p)','max(p)','count')

In [132]:
stat.show(truncate=False)

+-----+--------+--------+-----+
|id   |min(p)  |max(p)  |count|
+-----+--------+--------+-----+
|40340|361676  |16310104|653  |
|41157|1614507 |16231498|11   |
|40353|2965266 |14864548|865  |
|10801|16159427|16159427|1    |
|41286|14423289|14423289|1    |
|12067|1965369 |14646714|7    |
|44208|6595345 |11031077|3    |
|39470|10707296|10707296|1    |
|39368|3745307 |14548520|6    |
|39005|28924   |15179996|6    |
|39492|3533174 |16187880|2    |
|39512|11516335|11516335|1    |
|42208|4514087 |16300213|5    |
|42247|12812930|12812930|1    |
|40229|2488010 |2488010 |1    |
|41135|6272941 |16255071|2    |
|39493|1604638 |13437552|8    |
|41159|627562  |14385024|25   |
|41329|5237460 |9262095 |3    |
|44429|7680432 |7680432 |1    |
+-----+--------+--------+-----+
only showing top 20 rows



In [133]:
stat.sort('count',ascending=True).show(truncate=False)

+-----+--------+--------+-----+
|id   |min(p)  |max(p)  |count|
+-----+--------+--------+-----+
|40003|12740765|12740765|1    |
|10623|9347367 |9347367 |1    |
|40211|6776437 |6776437 |1    |
|10725|2037681 |2037681 |1    |
|40737|10284093|10284093|1    |
|41495|14752427|14752427|1    |
|44426|7526249 |7526249 |1    |
|50474|3081312 |3081312 |1    |
|12005|4908529 |4908529 |1    |
|10662|9006891 |9006891 |1    |
|44425|2713358 |2713358 |1    |
|11113|12562471|12562471|1    |
|39512|11516335|11516335|1    |
|10801|16159427|16159427|1    |
|41286|14423289|14423289|1    |
|41021|8742136 |8742136 |1    |
|41016|7525857 |7525857 |1    |
|39513|3039572 |3039572 |1    |
|43602|9932658 |9932658 |1    |
|44709|10374777|10374777|1    |
+-----+--------+--------+-----+
only showing top 20 rows



In [134]:
stat.sort('min(p)',ascending=True).show(truncate=False)

+-----+------+--------+-----+
|id   |min(p)|max(p)  |count|
+-----+------+--------+-----+
|40316|6216  |16775934|10014|
|40318|8216  |16267105|5257 |
|39494|9469  |14546794|7    |
|42245|18827 |14325084|6    |
|40308|19508 |16226533|34705|
|40338|27224 |16767053|3436 |
|39005|28924 |15179996|6    |
|40312|46942 |16771193|6869 |
|10024|48747 |48747   |1    |
|40334|51959 |16283325|6029 |
|42209|78184 |14339772|8    |
|10664|79046 |79046   |1    |
|39000|87943 |87943   |1    |
|40329|93923 |16748422|11823|
|39152|94791 |11501060|6    |
|44213|95891 |14914471|9    |
|41142|142545|14750374|3    |
|42234|176034|176034  |1    |
|40209|176829|16404522|64   |
|39521|193599|15110312|6    |
+-----+------+--------+-----+
only showing top 20 rows



In [135]:
stat.sort('max(p)',ascending=True).show(truncate=False)

+-----+-------+-------+-----+
|id   |min(p) |max(p) |count|
+-----+-------+-------+-----+
|10024|48747  |48747  |1    |
|10664|79046  |79046  |1    |
|39000|87943  |87943  |1    |
|42234|176034 |176034 |1    |
|40224|364918 |364918 |1    |
|40343|365383 |365383 |1    |
|39511|903733 |903733 |1    |
|11023|1560549|1560549|1    |
|10004|1610151|1610151|1    |
|42956|1667256|1667256|1    |
|41312|1830689|1830689|1    |
|41144|1832426|1832426|1    |
|40566|1926654|1926654|1    |
|12044|1955463|1955463|1    |
|40371|1989326|1989326|1    |
|39226|2032306|2032306|1    |
|10725|2037681|2037681|1    |
|11316|1824372|2072686|2    |
|39781|2484047|2484047|1    |
|40229|2488010|2488010|1    |
+-----+-------+-------+-----+
only showing top 20 rows



In [136]:
stat.dtypes

[('id', 'string'),
 ('min(p)', 'bigint'),
 ('max(p)', 'bigint'),
 ('count', 'bigint')]

In [137]:
stat = stat.withColumn('effect_density',col('count')/(col('max(p)')-col('min(p)')))

In [138]:
stat.show(truncate=False)

+-----+--------+--------+-----+---------------------+
|id   |min(p)  |max(p)  |count|effect_density       |
+-----+--------+--------+-----+---------------------+
|40340|361676  |16310104|653  |4.094447427671241E-5 |
|41157|1614507 |16231498|11   |7.525488659054384E-7 |
|40353|2965266 |14864548|865  |7.26934616727295E-5  |
|10801|16159427|16159427|1    |null                 |
|41286|14423289|14423289|1    |null                 |
|12067|1965369 |14646714|7    |5.519919219925016E-7 |
|44208|6595345 |11031077|3    |6.76325801468619E-7  |
|39470|10707296|10707296|1    |null                 |
|39368|3745307 |14548520|6    |5.553903269332929E-7 |
|39005|28924   |15179996|6    |3.960115825467663E-7 |
|39492|3533174 |16187880|2    |1.5804397194213758E-7|
|39512|11516335|11516335|1    |null                 |
|42208|4514087 |16300213|5    |4.24227604558105E-7  |
|42247|12812930|12812930|1    |null                 |
|40229|2488010 |2488010 |1    |null                 |
|41135|6272941 |16255071|2  

In [ ]:
minmaxscaler

In [105]:
inp.show(truncate=False)

+------------+------+------+----+----+
|mac         |m1    |m2    |p   |id  |
+------------+------+------+----+----+
|24DF6A0018CA|24DF6A|0018CA|6346|null|
|24DF6A0018CE|24DF6A|0018CE|6350|null|
+------------+------+------+----+----+



In [67]:
#xx = [{'p':'6350'}]
#xx = spark.createDataFrame(xx)
#xx.withColumn('m',conv(xx.p,10,16)).show(truncate=False)

/usr/lib/spark/python/pyspark/sql/session.py:357: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+----+----+
|p   |m   |
+----+----+
|6350|18CE|
+----+----+



In [135]:
#from pyspark.sql.window import Window
#window = Window.partitionBy().orderBy('index')
#combine.withColumn('shift_down', lag(combine.id,10).over(window)).withColumn('shift_up', lead(combine.id,10).over(window)).show(truncate=False)

+------------+------+------+----+-----+-----+----------+--------+
|mac         |m1    |m2    |p   |id   |index|shift_down|shift_up|
+------------+------+------+----+-----+-----+----------+--------+
|24DF6A001848|24DF6A|001848|6216|40316|0    |null      |40316   |
|24DF6A001856|24DF6A|001856|6230|40316|1    |null      |40316   |
|24DF6A00185E|24DF6A|00185E|6238|40316|2    |null      |40316   |
|24DF6A001866|24DF6A|001866|6246|40316|3    |null      |40316   |
|24DF6A001873|24DF6A|001873|6259|40316|4    |null      |null    |
|24DF6A001880|24DF6A|001880|6272|40316|5    |null      |40316   |
|24DF6A001884|24DF6A|001884|6276|40316|6    |null      |40316   |
|24DF6A001891|24DF6A|001891|6289|40316|7    |null      |40316   |
|24DF6A001892|24DF6A|001892|6290|40316|8    |null      |40316   |
|24DF6A001899|24DF6A|001899|6297|40316|9    |null      |40316   |
|24DF6A0018A3|24DF6A|0018A3|6307|40316|10   |40316     |40316   |
|24DF6A0018A7|24DF6A|0018A7|6311|40316|11   |40316     |40316   |
|24DF6A001

In [153]:
combine.

['mac', 'm1', 'm2', 'p', 'id', 'index']